<a href="https://colab.research.google.com/github/AadamHaq/Stock-Prediction-with-HMM/blob/main/HMM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Play around aa.us.txt

Just a simple sample code. Not for use but could take as reference.

In [ ]:
#Change path to google drive

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
#current path
!pwd

/gdrive


In [ ]:
#simply visualize
import pandas as pd

df1 = pd.read_csv("/gdrive/My Drive/Data/Stocks/aa.us.txt")
df1.tail()

,Date,Open,High,Low,Close,Volume,OpenInt
12069,2017-11-06,47.20,47.6800,46.53,47.29,1725638,0
12070,2017-11-07,47.11,47.2042,46.54,46.97,2168351,0
12071,2017-11-08,46.73,46.7700,45.37,45.89,3347930,0
12072,2017-11-09,44.71,44.7100,43.11,43.33,7671810,0
12073,2017-11-10,42.93,44.4600,42.75,43.01,4463839,0


In [ ]:
#search for one state
df1.loc[df1['Date'] == '1999-11-18']

,Date,Open,High,Low,Close,Volume,OpenInt
7550,1999-11-18,64.85,64.85,63.641,64.083,1057225,0


In [ ]:
df1 = df1[int(len(df1['Date'])-10000):len(df1['Date'])]
df1.reset_index(drop = True, inplace = True)
df1.head()

In [ ]:
len(df1)

10000

In [ ]:
data1 = df1[['Open','High','Low','Close']]
data1['pct_ret'] = data1['Close'].pct_change()
data1.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,High,Low,Close,pct_ret
9995,47.20,47.6800,46.53,47.29,0.003608
9996,47.11,47.2042,46.54,46.97,-0.006767
9997,46.73,46.7700,45.37,45.89,-0.022993
9998,44.71,44.7100,43.11,43.33,-0.055786
9999,42.93,44.4600,42.75,43.01,-0.007385


In [ ]:
data1['state'] = data1['pct_ret'].apply(lambda x:
                                        'up' if (x>0.001) \
                                        else('down' if (x<-0.001)\
                                             else 'no_change'))
data1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Open,High,Low,Close,pct_ret,state
0,2.6309,2.6309,2.5475,2.5475,NaN,no_change
1,2.5475,2.5475,2.4842,2.4842,-0.024848,down
2,2.4842,2.5058,2.4842,2.5058,0.008695,up
3,2.5058,2.5058,2.4635,2.4635,-0.016881,down
4,2.4635,2.5269,2.4635,2.5269,0.025736,up


In [ ]:
data1['prev_state'] = data1['state'].shift(1)
data1.head()

,Open,High,Low,Close,pct_ret,state,prev_state
0,2.6309,2.6309,2.5475,2.5475,NaN,no_change,NaN
1,2.5475,2.5475,2.4842,2.4842,-0.024848,down,no_change
2,2.4842,2.5058,2.4842,2.5058,0.008695,up,down
3,2.5058,2.5058,2.4635,2.4635,-0.016881,down,up
4,2.4635,2.5269,2.4635,2.5269,0.025736,up,down


In [ ]:
#state num matrix
state_space = data1[['state','prev_state']]
state_space_matrix = data1.groupby(['state','prev_state'])
state_space_matrix = state_space_matrix.size().unstack()
state_space_matrix

prev_state,down,no_change,up
state,,,
down,2261,291,2121
no_change,300,66,277
up,2111,287,2285


In [ ]:
#transition matrix
transition_matrix = state_space_matrix.apply(lambda x: x/int(len(data1)))
transition_matrix

prev_state,down,no_change,up
state,,,
down,0.2261,0.0291,0.2121
no_change,0.0300,0.0066,0.0277
up,0.2111,0.0287,0.2285
